In [2]:
from google.colab import drive
drive.mount('/content/drive')
save_path = "/content/drive/My Drive/mlex3"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import re
from pathlib import Path
import pandas as pd
import time
from pathlib import Path
import numpy as np
from tqdm import tqdm
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import Flatten, Activation
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input


Using TensorFlow backend.


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:

def get_images_and_labels(path, file_ending= "jpg"):
    paths = []
    labels = []
    for directory in Path(path).glob('*'):
        for p in directory.glob("*." + file_ending):
            paths.append(p)
            labels.append(directory.name)

    return paths, labels


fruit_image_paths, fruit_image_labels = get_images_and_labels("/content/drive/My Drive/data/FIDS30/")

traffic_sign_image_paths_train, traffic_sign_image_labels_train = get_images_and_labels("/content/drive/My Drive/data/TrafficSigns/train", "ppm")
traffic_sign_image_paths_test, traffic_sign_image_labels_test = get_images_and_labels("/content/drive/My Drive/data/TrafficSigns/test","ppm")

In [ ]:
def load_and_prepare_fruit_images(files, labels, augment = False, batch_size = 16  ): 
    
    np.random.seed(1234)
    target_size=(224, 224) # the same as the pretrained VGG net
    validation_size = 0.1
    test_size = 0.2

    data = pd.DataFrame(list(zip(files, labels)),columns=['filename', 'class'],dtype=np.str)
    train, test = train_test_split(data, test_size=test_size, random_state=1234)  
    
    if augment is True:
        image_generator = ImageDataGenerator(rescale=1/255.,
                                            validation_split=validation_size,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True
                                     )
    else:
        image_generator = ImageDataGenerator(rescale=1/255.,
                                             validation_split=validation_size)
    
    train_generator = image_generator.flow_from_dataframe(train, batch_size=batch_size,
                                            target_size=target_size,
                                            class_mode='categorical',
                                            subset='training')
    validation_generator = image_generator.flow_from_dataframe(train, batch_size=batch_size,
                                          target_size=target_size,
                                          class_mode='categorical',
                                          subset='validation')

    image_generator = ImageDataGenerator(rescale=1/255.,)
    

    test_generator = image_generator.flow_from_dataframe(test, batch_size=batch_size,
                                           target_size=target_size,
                                           class_mode='categorical',
                                           shuffle=False)

    return train_generator, test_generator, validation_generator

In [7]:
train_generator, test_generator, validation_generator = load_and_prepare_fruit_images(fruit_image_paths, fruit_image_labels)


Found 699 validated image filenames belonging to 30 classes.
Found 77 validated image filenames belonging to 30 classes.
Found 195 validated image filenames belonging to 30 classes.


In [ ]:
def createVGGNetSelf(input_shape, classes ):
   
    model = Sequential()

    # Layer 1
    model.add(Conv2D(32, (3, 3), input_shape=input_shape, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    # Layer 2
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Layer 3
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Full Layer
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Classifier
    model.add(Dense(classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    return model


def createVGGNet(input_shape, classes ):
   
    model = VGG16(include_top=True, weights=None, input_shape=input_shape, classes=classes)
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    return model

def createVGGNetPreTrained(input_shape, classes ):
   
    np.random.seed(1234)
    pretrained_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in pretrained_model.layers:
        layer.trainable = False
    
    y = Flatten()(pretrained_model.output)
    y = Dense(500, activation="relu")(y)
    y = Dropout(0.2)(y)
    y = Dense(1000, activation="relu")(y)
    y = Dense(classes, activation='softmax', name='my_dense_2')(y)
    model = Model( pretrained_model.input, y)

    
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    return model


We are using a VGG16 deeplearning architecture pretrained on the imagenet dataset. 
We adapted this architecture to fit our data set/classes. See https://towardsdatascience.com/step-by-step-guide-to-using-pretrained-models-in-keras-c9097b647b29 for how to do this. This allows us to train our classifier much faster since we just train the last added Dense layer. 

In [ ]:
def plotConfusionMatrix(y_true, y_pred, classes, name=None, normalize=True,
                          cmap=plt.cm.Blues):

    cm = confusion_matrix(y_true, y_pred)
    classes = np.array(list(classes))[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title='Confusion Matrix',
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    size = 6 if normalize else 9
    thresh = 0.5 if normalize else cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center", size=size,
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()

    if name is not None:
        plt.savefig(save_path+'/figures/' + name + '_cm.png', dpi=150)
        plt.close()


def plotTrainingCurve(history, name=None):
    sns.set_style('ticks')

    hist = history.history

    plt.figure(figsize=(10,6))
    plt.title("Training Curve")
    plt.xlabel("Epoch")

    for measure in hist.keys():
        epochs = range(1, len(hist[measure]) + 1)
        plt.plot(epochs, hist[measure], label=measure)

    plt.xticks(epochs)

    plt.legend()
    sns.despine()
    plt.tight_layout()

    if name is not None:
        plt.savefig(save_path+'/figures/' + name + '.png')
        plt.close()

# Train model on non augmented data

In [10]:
 
input_shape = train_generator.image_shape
model = createVGGNetPreTrained(input_shape, len(np.unique(fruit_image_labels)))

start_time = time.time()
model_history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator),
                                  validation_data=validation_generator,
                                  validation_steps=len(validation_generator),
                                epochs=10)
training_duration = time.time() - start_time

model.save(save_path+'/models/fruit_model.h5')





58892288/58889256 [==============================] - 1s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
44/44 [==============================] - 191s 4s/step - loss: 3.3040 - acc: 0.1569 - val_loss: 2.7019 - val_acc: 0.3636
Epoch 2/10
44/44 [==============================] - 19s 430ms/step - loss: 1.8080 - acc: 0.4770 - val_loss: 1.8127 - val_acc: 0.4805
Epoch 3/10
44/44 [==============================] - 20s 457ms/step - loss: 1.0745 - acc: 0.6721 - val_loss: 1.8938 - val_acc: 0.4416
Epoch 4/10
44/44 [==============================] - 20s 458ms/step - loss: 0.6967 - acc: 0.7877 - val_loss: 1.5230 - val_acc: 0.5584
Epoch 5/10
44/44 [==============================] - 20s 449ms/step - loss: 0.4175 - acc: 0.8709 - val_loss: 1.7086 - val_acc: 0.5455
Epoch 6/10
44/44 [==============================

# Test non augmented data

In [ ]:
summary = {}
model_name = "Fruits_non_augmented"

start_time = time.time()
predictions = model.predict_generator(test_generator, steps=len(test_generator))
prediction_duration =  time.time() - start_time

score = accuracy_score(test_generator.classes, predictions.argmax(axis=1))

train_data = {}
train_data['accuracy_score'] = score
train_data['train_time'] = training_duration
train_data['train_time'] = prediction_duration
summary[model_name] = train_data

plotTrainingCurve(model_history, name=model_name)
plotConfusionMatrix(test_generator.classes, predictions.argmax(axis=1),
                    train_generator.class_indices, name=model_name)

# Train model on augmented data

In [12]:
train_generator_augmented, test_generator_augmented, validation_generator_augmented = load_and_prepare_fruit_images(fruit_image_paths, fruit_image_labels, augment=True)

Found 699 validated image filenames belonging to 30 classes.
Found 77 validated image filenames belonging to 30 classes.
Found 195 validated image filenames belonging to 30 classes.


In [13]:

input_shape = train_generator_augmented.image_shape
model_augmented = createVGGNetPreTrained(input_shape, len(np.unique(fruit_image_labels)))

start_time = time.time()
model_augmented_history = model_augmented.fit_generator(train_generator_augmented, steps_per_epoch=len(train_generator_augmented),
                                  validation_data=validation_generator_augmented,
                                  validation_steps=len(validation_generator_augmented),
                                epochs=10)
training_duration = time.time() - start_time


model_augmented.save(save_path+'/models/fruit_model_augmented.h5')

Epoch 1/10
44/44 [==============================] - 28s 632ms/step - loss: 3.9191 - acc: 0.0936 - val_loss: 2.9314 - val_acc: 0.1039
Epoch 2/10
44/44 [==============================] - 27s 615ms/step - loss: 2.6936 - acc: 0.2101 - val_loss: 2.5290 - val_acc: 0.3377
Epoch 3/10
44/44 [==============================] - 27s 606ms/step - loss: 2.3176 - acc: 0.2966 - val_loss: 2.4754 - val_acc: 0.2987
Epoch 4/10
44/44 [==============================] - 27s 614ms/step - loss: 2.0717 - acc: 0.3675 - val_loss: 2.2096 - val_acc: 0.3506
Epoch 5/10
44/44 [==============================] - 27s 612ms/step - loss: 1.9168 - acc: 0.4189 - val_loss: 1.9201 - val_acc: 0.4416
Epoch 6/10
44/44 [==============================] - 27s 610ms/step - loss: 1.7562 - acc: 0.4428 - val_loss: 1.9108 - val_acc: 0.4545
Epoch 7/10
44/44 [==============================] - 27s 619ms/step - loss: 1.6850 - acc: 0.4732 - val_loss: 1.8446 - val_acc: 0.4675
Epoch 8/10
44/44 [==============================] - 27s 612ms/step - 

# Test on augmented data

In [ ]:
model_name = "Fruits_augmented"

start_time = time.time()
predictions = model_augmented.predict_generator(test_generator_augmented, steps=len(test_generator_augmented))
prediction_duration =  time.time() - start_time

score = accuracy_score(test_generator_augmented.classes, predictions.argmax(axis=1))

train_data = {}
train_data['accuracy_score'] = score
train_data['train_time'] = training_duration
train_data['train_time'] = prediction_duration
summary[model_name] = train_data

plotTrainingCurve(model_history, name=model_name)
plotConfusionMatrix(test_generator.classes, predictions.argmax(axis=1),
                    train_generator.class_indices, name=model_name)

In [ ]:

summary_df = pd.DataFrame.from_dict(summary, orient='index')
summary_df.to_csv(save_path+'/scores/model_fruits.txt')

In [16]:
display(summary_df)

,accuracy_score,train_time
Fruits_non_augmented,0.482051,51.031782
Fruits_augmented,0.507692,6.390636


We know that the imagenet dataset contains a lof of different classes including fruits. Thus it is alsready very good a predicting fruit images and thus the good results 

# Traffic Sign Data

For the traffic sign data here we don't flip the augemented images, since in real world don't observe flipped signs. 

In [ ]:
def load_and_prepare_traffic_images(files, labels, augment = False, batch_size = 16  ): 
    
    np.random.seed(1234)
    target_size=(224, 224) # the same as the pretrained VGG net
    validation_size = 0.1
    test_size = 0.2

    data = pd.DataFrame(list(zip(files, labels)),columns=['filename', 'class'],dtype=np.str)
    train, test = train_test_split(data, test_size=test_size, random_state=1234)  
    
    if augment is True:
        image_generator = ImageDataGenerator(rescale=1/255.,
                                            validation_split=validation_size,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True)
    else:
        image_generator = ImageDataGenerator(rescale=1/255.,
                                             validation_split=validation_size)
    
    train_generator = image_generator.flow_from_dataframe(train, batch_size=batch_size,
                                            target_size=target_size,
                                            class_mode='categorical',
                                            subset='training')
    validation_generator = image_generator.flow_from_dataframe(train, batch_size=batch_size,
                                          target_size=target_size,
                                          class_mode='categorical',
                                          subset='validation')

    image_generator = ImageDataGenerator(rescale=1/255.,)
    

    test_generator = image_generator.flow_from_dataframe(test, batch_size=batch_size,
                                           target_size=target_size,
                                           class_mode='categorical',
                                           shuffle=False)

    return train_generator, test_generator, validation_generator

In [18]:
train_generator, test_generator, validation_generator = load_and_prepare_traffic_images(
    traffic_sign_image_paths_train+traffic_sign_image_paths_test, 
    traffic_sign_image_labels_train+traffic_sign_image_labels_test)

train_generator_augmented, test_generator_augmented, validation_generator_augmented = load_and_prepare_traffic_images(
    traffic_sign_image_paths_train+traffic_sign_image_paths_test, 
    traffic_sign_image_labels_train+traffic_sign_image_labels_test,
    augment = True)

Found 4362 validated image filenames belonging to 9 classes.
Found 484 validated image filenames belonging to 9 classes.
Found 1212 validated image filenames belonging to 9 classes.
Found 4362 validated image filenames belonging to 9 classes.
Found 484 validated image filenames belonging to 9 classes.
Found 1212 validated image filenames belonging to 9 classes.


# Train Traffic sign on non augmented data. 

In [19]:
 
input_shape = train_generator.image_shape
model = createVGGNetPreTrained(input_shape, len(np.unique(traffic_sign_image_labels_train+traffic_sign_image_labels_test)))

start_time = time.time()
model_history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator),
                                  validation_data=validation_generator,
                                  validation_steps=len(validation_generator),
                                epochs=10)
training_duration = time.time() - start_time

model.save(save_path+'/models/traffic_model.h5')

Epoch 1/10
273/273 [==============================] - 1314s 5s/step - loss: 0.3579 - acc: 0.8958 - val_loss: 0.2127 - val_acc: 0.9215
Epoch 2/10
273/273 [==============================] - 25s 91ms/step - loss: 0.1424 - acc: 0.9575 - val_loss: 0.0926 - val_acc: 0.9669
Epoch 3/10
273/273 [==============================] - 25s 93ms/step - loss: 0.1141 - acc: 0.9658 - val_loss: 0.0312 - val_acc: 0.9959
Epoch 4/10
273/273 [==============================] - 26s 95ms/step - loss: 0.0693 - acc: 0.9789 - val_loss: 0.0916 - val_acc: 0.9669
Epoch 5/10
273/273 [==============================] - 26s 97ms/step - loss: 0.1274 - acc: 0.9636 - val_loss: 0.0546 - val_acc: 0.9855
Epoch 6/10
273/273 [==============================] - 27s 97ms/step - loss: 0.0857 - acc: 0.9746 - val_loss: 0.0365 - val_acc: 0.9855
Epoch 7/10
273/273 [==============================] - 26s 96ms/step - loss: 0.0862 - acc: 0.9712 - val_loss: 0.0142 - val_acc: 0.9917
Epoch 8/10
273/273 [==============================] - 26s 97ms

Test on non Augmented Model 

In [ ]:
summary = {}
model_name = "Traffic_Signs_non_augmented"

start_time = time.time()
predictions = model.predict_generator(test_generator, steps=len(test_generator))
prediction_duration =  time.time() - start_time

score = accuracy_score(test_generator.classes, predictions.argmax(axis=1))

train_data = {}
train_data['accuracy_score'] = score
train_data['train_time'] = training_duration
train_data['train_time'] = prediction_duration
summary[model_name] = train_data

plotTrainingCurve(model_history, name=model_name)
plotConfusionMatrix(test_generator.classes, predictions.argmax(axis=1),
                    train_generator.class_indices, name=model_name)

# Train on augmented data

In [21]:

input_shape = train_generator_augmented.image_shape
model_augmented = createVGGNetPreTrained(input_shape, len(np.unique(traffic_sign_image_labels_train+traffic_sign_image_labels_test)))

start_time = time.time()
model_augmented_history = model_augmented.fit_generator(train_generator_augmented, steps_per_epoch=len(train_generator_augmented),
                                  validation_data=validation_generator_augmented,
                                  validation_steps=len(validation_generator_augmented),
                                epochs=10)
training_duration = time.time() - start_time


model_augmented.save(save_path+'/models/traffic_signs_model_augmented.h5')

Epoch 1/10
273/273 [==============================] - 60s 219ms/step - loss: 0.6315 - acc: 0.8004 - val_loss: 0.3466 - val_acc: 0.8802
Epoch 2/10
273/273 [==============================] - 58s 211ms/step - loss: 0.3889 - acc: 0.8757 - val_loss: 0.2768 - val_acc: 0.9029
Epoch 3/10
273/273 [==============================] - 57s 210ms/step - loss: 0.3341 - acc: 0.8929 - val_loss: 0.2530 - val_acc: 0.9153
Epoch 4/10
273/273 [==============================] - 57s 210ms/step - loss: 0.2906 - acc: 0.9065 - val_loss: 0.1549 - val_acc: 0.9463
Epoch 5/10
273/273 [==============================] - 56s 207ms/step - loss: 0.2729 - acc: 0.9132 - val_loss: 0.1896 - val_acc: 0.9442
Epoch 6/10
273/273 [==============================] - 57s 207ms/step - loss: 0.2718 - acc: 0.9123 - val_loss: 0.2119 - val_acc: 0.9298
Epoch 7/10
273/273 [==============================] - 57s 207ms/step - loss: 0.2593 - acc: 0.9084 - val_loss: 0.1916 - val_acc: 0.9277
Epoch 8/10
273/273 [==============================] - 5

# Test on Augmented Data


In [ ]:
model_name = "Traffic_Signs_Augmented"

start_time = time.time()
predictions = model_augmented.predict_generator(test_generator_augmented, steps=len(test_generator_augmented))
prediction_duration =  time.time() - start_time

score = accuracy_score(test_generator_augmented.classes, predictions.argmax(axis=1))

train_data = {}
train_data['accuracy_score'] = score
train_data['train_time'] = training_duration
train_data['train_time'] = prediction_duration
summary[model_name] = train_data

plotTrainingCurve(model_augmented_history, name=model_name)
plotConfusionMatrix(test_generator.classes, predictions.argmax(axis=1),
                    train_generator.class_indices, name=model_name)

In [ ]:

summary_df = pd.DataFrame.from_dict(summary, orient='index')
summary_df.to_csv(save_path+'/scores/models_traffic_signs.txt')

In [24]:
display(summary_df)

,accuracy_score,train_time
Traffic_Signs_non_augmented,0.995050,412.113385
Traffic_Signs_Augmented,0.971122,6.864012
